# Spark DataFrame Basics III - GroupBy and Aggregate functions

<p>Obs.: After download the databricks notebook to .ipynb we have problems in the output format but if you run this notebook in a databricks cluster you'll have a output in a table format.</p>

<p>E.g.:</p>
<p>The following output:</p>
<p>+----+-------+ age| name| +----+-------+ null|Michael| 30| Andy| 19| Justin| +----+-------+</p>
<p>actually is:</p>
<pre>+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+  </pre>

### Create session and load data

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [3]:
df = spark.read.csv('/FileStore/tables/sales_info.csv', inferSchema=True, header=True)

In [4]:
df.show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Tina|600.0|
 MSFT| Amy|124.0|
 MSFT|Vanessa|243.0|
 FB| Carl|870.0|
 FB| Sarah|350.0|
 APPL| John|250.0|
 APPL| Linda|130.0|
 APPL| Mike|750.0|
 APPL| Chris|350.0|
+-------+-------+-----+

In [5]:
df.printSchema()

root
-- Company: string (nullable = true)
-- Person: string (nullable = true)
-- Sales: double (nullable = true)

### Use groupBy function

In [6]:
df.groupBy('Company')

Out[ 6 ]: <pyspark.sql.group.GroupedData at 0x7f3141130c88>

In [7]:
df.groupBy('Company').mean().show()

+-------+-----------------+
Company| avg(Sales)|
+-------+-----------------+
 APPL| 370.0|
 GOOG| 220.0|
 FB| 610.0|
 MSFT|322.3333333333333|
+-------+-----------------+

### Use aggregate function

In [8]:
df.agg({'Sales': 'sum'}).show()

+----------+
sum(Sales)|
+----------+
 4327.0|
+----------+

In [9]:
group_data = df.groupBy('Company')

In [10]:
group_data.agg({'Sales': 'max'}).show()

+-------+----------+
Company|max(Sales)|
+-------+----------+
 APPL| 750.0|
 GOOG| 340.0|
 FB| 870.0|
 MSFT| 600.0|
+-------+----------+

### Use pyspark functions for descriptive statistics analysis of the data

In [11]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [12]:
df.select(countDistinct('Sales')).show()

+---------------------+
count(DISTINCT Sales)|
+---------------------+
 11|
+---------------------+

In [13]:
df.select(avg('Sales').alias('Average Sales')).show()

+-----------------+
 Average Sales|
+-----------------+
360.5833333333333|
+-----------------+

In [14]:
df.select(stddev('Sales')).show()

+------------------+
stddev_samp(Sales)|
+------------------+
250.08742410799007|
+------------------+

### Use format_number function

In [15]:
from pyspark.sql.functions import format_number

In [16]:
sales_std = df.select(stddev('Sales').alias('std'))
sales_std.select(format_number('std', 2).alias('std')).show()

+------+
 std|
+------+
250.09|
+------+

### Use orderBy function

In [17]:
df.orderBy('Sales').show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG|Charlie|120.0|
 MSFT| Amy|124.0|
 APPL| Linda|130.0|
 GOOG| Sam|200.0|
 MSFT|Vanessa|243.0|
 APPL| John|250.0|
 GOOG| Frank|340.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 MSFT| Tina|600.0|
 APPL| Mike|750.0|
 FB| Carl|870.0|
+-------+-------+-----+

In [18]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 FB| Carl|870.0|
 APPL| Mike|750.0|
 MSFT| Tina|600.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 GOOG| Frank|340.0|
 APPL| John|250.0|
 MSFT|Vanessa|243.0|
 GOOG| Sam|200.0|
 APPL| Linda|130.0|
 MSFT| Amy|124.0|
 GOOG|Charlie|120.0|
+-------+-------+-----+